# Парсинг данных

Наша команда не хотела брать готовый парсер, поскольку мы опасались не понимать работу кода. По этой причине мы решили парсить самостоятельно.
Изначально для парсинга был выбран сайт ЦИАН, но, тщательно изучив объявления на нем, наша команда выяснила, что у этого сайта достаточно неупорядоченный интерфейс. Спарсив его, мы бы столкнулись с проблемой огромного количества пропуска данных, что несомненно сказалось бы на качестве работы модели. Но команда нашла альтернативный сайт по продаже недвижимости - Авито. Он обладает более понятной и упорядоченной структурой, что помогает преодолеть трудность в виде пропусков. По этой причине мы сменили источник. Более того, у Авито более понятная html-структура страницы.

In [ ]:
!pip install selenium

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
url = "https://www.avito.ru/moskva/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?f=ASgBAQICAUSSA8YQAkDmBxSMUsoIpIpZmqwBmKwBlqwBlKwBiFmGWYRZglmAWQ"
driver = webdriver.Chrome()

In [ ]:
from selenium.webdriver.common.by import By
driver.get(url)

In [ ]:
all_links = []
num_pages = 25
for i in range(1, num_pages):
    flats_one_page = driver.find_elements(By.TAG_NAME,'a')
    for flat in flats_one_page:
        all_links.append(flat.get_attribute("href"))
        print(flat.get_attribute("href"))    
    print(i)
    driver.find_element("xpath", "//*[@id='app']/div/div[3]/div/div[2]/div[3]/div[3]/div[5]/nav/ul/li[9]/a").click()

Сначала мы собираем абсолютно все ссылки с каждой страницы

In [ ]:
PATTERN_1 = 'https://www.avito.ru/moskva/kvartiry/'
PATTERN_2 = "-k._kvartira_"
with open("flat_links1.txt", "w" ) as file:
    for url in all_links:
        if url != None:
            if url.__contains__(PATTERN_1) and  url.__contains__(PATTERN_2):
                print(url, file = file)

Затем по PATTERN_1, PATTERN_2 ищем именно объявления о продаже квартир вторичного жилья и сохраняем в "flat_links1.txt". Однако при просмотре текстового файла выяснили, что одна и та же ссылка записывается 2 раза. Решаем эту проблему ниже.

In [ ]:
with open("flat_links1.txt", "r" ) as file:
    with open("flat_links2.txt", "w" ) as infile:
        i = 0
        for line in file:
            if i % 2 == 0:
                print(line, file = infile, end = "")
            i += 1

Данный файл создается по следующей причине: при каждой новой имитации входа человека на сайт Авито, он обновляет объявления на всех страницах. Так как наша команда работает на нескольких компьютерах одновременно, с целью оптимизации нашей работы мы создали текстовый документ, в который записали все необходимые нам объявления на каждой странице. Тем самым мы решаем проблему записи одинаковых данных на разных компьютерах. 

Получили текстовый файл "flat_links2.txt", где на каждой строчке находятся ссылки на необходимые нам объявления.
Поскольку при запуске кода на одном компьютере данные парсились очень долго, решили преобразовать код, чтобы иметь возможность запустить код в ячейке ниже на разных компьютерах (21 штука) одновременно, "делегируя" обязанности между ними. А именно принцип был следующий: первый компьютер парсит данные с 1 строки по 50 и сохраняет полученные данные в our_csv1.csv, второй - с 50 по 100 и сохраняет полученные данные в our_csv2.csv и т.д. Это позволяет ускорить код.  

In [ ]:
with open("flat_links2.txt", "r") as file:
    with open("our_csv3.csv", "w", encoding='UTF-8') as infile:
        print("Metro; Number of Rooms; Square; Current Floor; Total Floors; Auction; Price; Requisites Checked", end = "\n", file = infile)
        lines = file.readlines()
        for i in range(100, 150):
            link = lines[i]
            driver.get(link)
            try:
                metro = driver.find_element("xpath", "//*[@id='app']/div/div[2]/div[1]/div[2]/div[4]/div[1]/div[2]/div[2]/div/div[2]/div[1]/div/div/span/span[1]/span[2]").text
                print(metro, end = ";", file = infile) 
                string = driver.find_element(By.CLASS_NAME, "title-info-title-text").text
                
                if string.find("Аукцион") == -1:
                    lst = string.split()     
                    number_rooms = lst[0].split("-")[0]
                    print(number_rooms, end = ";", file = infile)  
                    square = lst[2]   
                    print(square, end = ";", file = infile)  
                    current_floor = lst[4].split("/")[0]
                    print(current_floor, end = ";", file = infile)   
                    total_floors = lst[4].split("/")[1]   
                    print(total_floors, end = ";", file = infile)
                    print('Нет аукциона', end = ";", file = infile)
                else:
                    lst = string.split()     
                    number_rooms = lst[1].split("-")[0]
                    print(number_rooms, end = ";", file = infile)  
                    square = lst[3]   
                    print(square, end = ";", file = infile) 
                    current_floor = lst[5].split("/")[0]
                    print(current_floor, end = ";", file = infile)   
                    total_floors = lst[5].split("/")[1]   
                    print(total_floors, end = ";", file = infile)
                    print('Аукцион', end = ";", file = infile)
                    
                price = driver.find_element(By.CLASS_NAME, "styles-module-size_xxxl-A2qfi").text
                print(price, end = ";", file = infile)  
                try:   
                    requisites_checked = driver.find_element(By.CLASS_NAME, "SnippetBadge-title-oSImJ").text
                    print(requisites_checked, end = "\n", file = infile)
                except:     
                    requisites_checked = "Реквизиты не проверены"
                    print(requisites_checked, end = "\n", file = infile)
            except:   
                try:
                    metro = driver.find_element("xpath", "//*[@id='app']/div/div[2]/div[1]/div[2]/div[3]/div[1]/div[2]/div[2]/div/div[2]/div[1]/div/div/span/span[1]/span[2]").text                  
                    print(metro, end = ";", file = infile) 
                    string = driver.find_element(By.CLASS_NAME, "title-info-title-text").text
                    
                    if string.find("Аукцион") == -1:
                        lst = string.split()     
                        number_rooms = lst[0].split("-")[0]
                        print(number_rooms, end = ";", file = infile)  
                        square = lst[2]   
                        print(square, end = ";", file = infile)  
                        current_floor = lst[4].split("/")[0]
                        print(current_floor, end = ";", file = infile)   
                        total_floors = lst[4].split("/")[1]   
                        print(total_floors, end = ";", file = infile)
                        print('Нет аукциона', end = ";", file = infile)
                    else:
                        lst = string.split()     
                        number_rooms = lst[1].split("-")[0]
                        print(number_rooms, end = ";", file = infile)  
                        square = lst[3]   
                        print(square, end = ";", file = infile)  
                        current_floor = lst[5].split("/")[0]
                        print(current_floor, end = ";", file = infile)
                        total_floors = lst[5].split("/")[1]   
                        print(total_floors, end = ";", file = infile)
                        print('Аукцион', end = ";", file = infile)
                    price = driver.find_element(By.CLASS_NAME, "styles-module-size_xxxl-A2qfi").text
                    print(price, end = ";", file = infile)
                    try:   
                        requisites_checked = driver.find_element(By.CLASS_NAME, "SnippetBadge-title-oSImJ").text
                        print(requisites_checked, end = "\n", file = infile)
                    except:     
                        requisites_checked = "Реквизиты не проверены"
                        print(requisites_checked, end = "\n", file = infile)
                except: 
                    continue 

В данном коде мы открываем текстовый файл "flat_links2.txt" с ссылками на объявления, проходимся циклом for по ссылкам и сразу записываем полученные признаки в файл csv. 

Нами были выбраны признаки "Metro" - станция метро, 'Number of Rooms' - количество комнат, "Square" - общая площадь; "Current Floor" - этаж, на котором находится квартира; "Total Floors" - общее количество этажей в доме; "Auction" - наличие аукциона; "Requisites Checked" - наличие проверенных реквизитов и "Price" - цена, наша целевая переменная.

Мы разграничили код по переменной "Auction", поскольку когда квартира находилась на аукционе, то менялся паттерн, по которому происходило сплитование строки (if-else). 

Также было 2 возможных xpath для метро в зависимости от типа объявления (try-except). А также мы обходили случай, когда станция метро не указана в объявлении путем использования вложенного try-except. Такие объявления не рассматривались, так как путем анализа нескольких из них, выяснили, что они находятся не в Москве, а в Подмосковье, несмотря на установленные фильтры на сайте. 